[SOI = Sphere of Influence](https://en.wikipedia.org/wiki/Sphere_of_influence_(astrodynamics)), with radius 
$$
    r_\mathrm{SOI} \coloneqq a (m/M_\odot)^{2/5}
$$

In [127]:
from astropy.time import Time
import spiceypy as sp

KM2AU = sp.convrt(1, "km", "au")
RAD2DEG = sp.dpr()
SEC2YR = sp.convrt(1, "seconds", "years")


def gfdist():
    pass


# Load the SPICE kernel meta file
sp.furnsh('kernel_meta.txt')

# Get the G*M value of the Sun and Jupiter
GM_SUN = sp.bodvcd(bodyid=10, item='GM', maxn=1)[1][0]
GM_JUP = sp.bodvcd(bodyid=5, item='GM', maxn=1)[1][0]

# Set initial time
T0_UTC = Time("2000-01-01T12:00:00", format='isot', scale='utc')  # Random time
T1_UTC = Time("2004-01-01", format='isot', scale='utc')  # Start of ephemeris of 67P
T2_UTC = Time("2015-12-31", format='isot', scale='utc')  # End of ephemeris of 67P
T0_ET = sp.utc2et(T0_UTC.utc.iso)
T1_ET = sp.utc2et(T1_UTC.utc.iso)
T2_ET = sp.utc2et(T2_UTC.utc.iso)

# SOI of Jupiter
vec_jup, _lt = sp.spkgeo(targ=5, et=T0_ET, ref='ECLIPJ2000', obs=10)
elt_jup = sp.oscltx(vec_jup, T0_ET, GM_SUN)
R_SOI_JUP = elt_jup[-2]*(GM_JUP/GM_SUN)**(2/5)
print("SOI Radius of Jupiter:", R_SOI_JUP*KM2AU, "au")

# Init state of 67P
vec_67p, _lt = sp.spkgeo(targ=1000012, et=T1_ET, ref='ECLIPJ2000', obs=10)
elt_67p = sp.oscltx(vec_67p, T1_ET, GM_SUN)

SOI Radius of Jupiter: 0.32268429545253885 au


Since the kernel ``67P_CHURY_GERAS_2004_2016.BSP`` covers 2004-2016, one cannot calculate beyond that.

* e.g., ``sp.spkgeo(targ=1000012, et=T2_ET+(365*86400), ref='ECLIPJ2000', obs=10)`` will raise an "insufficient ephemeris" error.

For example, times when 67P is within ``D < 20*R_SOI_JUP``(≈6.44au) can easily be found:

In [142]:
cnfine = sp.cell_double(200)
sp.wninsd(T1_ET, T2_ET, cnfine)

result = sp.gfdist(target="1000012", abcorr="None", obsrvr="5", relate="<", refval=20*R_SOI_JUP,
                   adjust=0.0, step=3600., nintvls=100, cnfine=cnfine)
# Print the result
for i in range(0, sp.wncard(result)):
    interval = sp.wnfetd(result, i)
    print('Start time: ', sp.timout(interval[0], 'YYYY-MM-DD HR:MN:SC.###'))
    print('End time  : ', sp.timout(interval[1], 'YYYY-MM-DD HR:MN:SC.###'))

Start time:  2004-01-01 00:00:00.000
End time  :  2009-04-30 23:14:34.572
Start time:  2015-06-13 13:42:53.165
End time  :  2015-12-31 00:00:00.000


However, if you increase the time range outside 2004-2016 (e.g., ``sp.wninsd(T1_ET, T2_ET + 86400, cnfine)``), it will return an **insufficient ephemeris error**:

Thus, for future events, one has to calculate the trajectory manually in python...

In the original notebook, if TSOI == "time when 67P touches the ``R_SOI_JUP``", they did 

1. Assume the orbital element has been constant since 2000, find TSOI.
2. After TSOI, conduct a 2-body calculation using ``mu=GM_Jup`` until it goes out of the SOI again.
3. Convert the last state vector into an orbital element and compare it with the elements in 2000.

However, this is a very crude approximation, which maybe educationally useful, but scientifically not very useful. If you need an accurate trajectory, full calculation is needed and python for/while loop is not the best option (if you have to do this for millions of asteroids).